In [1]:
import sys
sys.path.append("..")

In [2]:
from tqdm.notebook import tqdm

from mlp_mixer.dataset.iterator import ImageNetIterator
from gmlp.layer.model import Gmlp
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
train_iter = ImageNetIterator(root='../../imagenet/',
                 height=256,
                 width=256,
                 is_train=True,
                 in_memory=False,
                 verbose=False)

test_iter = ImageNetIterator(root='../../imagenet/',
                 height=256,
                 width=256,
                 is_train=True,
                 in_memory=False,
                 verbose=False)

train_loader = DataLoader(train_iter, batch_size=64*2, shuffle=True, num_workers=10)
valid_loader = DataLoader(test_iter, batch_size=64*2, shuffle=False, num_workers=10)

In [4]:
h, w, c = 256, 256, 3
p = 32
s = (h*w)//(p**2)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = nn.DataParallel(Gmlp(h, w, c, p, s, 128, 128*4, 6, len(train_iter.label_dict)).to(device))

In [6]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [7]:
# torch.autograd.set_detect_anomaly(True)

def train() : 
    model.train()
    losses = []
    accuracies = []

    for x,y in tqdm(train_loader, desc='train') : 
        pred = model(x.to(device))

        optimizer.zero_grad()
        loss = criterion(pred, y.to(device))
        loss.backward()
        optimizer.step()

        correct = (torch.argmax(pred, dim=1) == y.to(device)).sum()
        acc = correct.item() / y.shape[0]

        accuracies.append(acc)
        losses.append(loss.item())

    agg_acc = sum(accuracies) / len(accuracies)
    agg_loss = sum(losses) / len(losses)
    return agg_acc, agg_loss

def evalulate() : 
    model.eval()
    losses = []
    accuracies = []

    for x,y in tqdm(valid_loader, desc='valid') : 
        pred = model(x.to(device))

        loss = criterion(pred, y.to(device))
        correct = (torch.argmax(pred, dim=1) == y.to(device)).sum()
        acc = correct.item() / y.shape[0]

        accuracies.append(acc)
        losses.append(loss.item())

    agg_acc = sum(accuracies) / len(accuracies)
    agg_loss = sum(losses) / len(losses)
    return agg_acc, agg_loss

In [8]:
epoches = 50

for proc in range(epoches) : 
    t_acc, t_loss = train()
    v_acc, v_loss = evalulate()
    print(f"""
                            === {proc+1}th Epoch ===
    
        Train Loss : {round(t_loss, 3)} | Train Acc : {round(t_acc, 3)}
        Valid Loss : {round(v_loss, 3)} | Valid Acc : {round(v_acc, 3)}
        
        ============================================
        ============================================
    """)

train:   0%|          | 0/600 [00:00<?, ?it/s]

valid:   0%|          | 0/600 [00:00<?, ?it/s]


                            === 1th Epoch ===
    
        Train Loss : 50.954 | Train Acc : 0.009
        Valid Loss : 5.238 | Valid Acc : 0.01
        
    


train:   0%|          | 0/600 [00:00<?, ?it/s]

valid:   0%|          | 0/600 [00:00<?, ?it/s]

...

valid:   0%|          | 0/600 [00:00<?, ?it/s]


                            === 49th Epoch ===
    
        Train Loss : 2.589 | Train Acc : 0.356
        Valid Loss : 2.498 | Valid Acc : 0.376
        
    


train:   0%|          | 0/600 [00:00<?, ?it/s]

valid:   0%|          | 0/600 [00:00<?, ?it/s]


                            === 50th Epoch ===
    
        Train Loss : 2.57 | Train Acc : 0.361
        Valid Loss : 2.499 | Valid Acc : 0.375
        
    
